In [ ]:
from pynq import Overlay, allocate
import pynq.lib.dma
import time, random, numpy

ol = Overlay('./design_1.bit')
ol.download()

CONST0_ADDRESS = 0x10
CONST1_ADDRESS = 0x18
ip = ol.ssmul_0
dma0 = ol.axi_dma_0
dma1 = ol.axi_dma_1

length = 1000000

in_buffer0 = allocate(shape=(length,), dtype=numpy.int32) 
in_buffer1 = allocate(shape=(length,), dtype=numpy.int32)
out_buffer_silicon = allocate(shape=(length,), dtype=numpy.int32) 

samples0 = random.sample(range(0, length), length)
samples1 = random.sample(range(0, length), length)
zeros = numpy.zeros((length,), dtype=int)

numpy.copyto(in_buffer0, samples0)
numpy.copyto(in_buffer1, samples1)
numpy.copyto(out_buffer_silicon, zeros)

# On Silicon

In [ ]:
ip.write(CONST0_ADDRESS, 1)
ip.write(CONST1_ADDRESS, 1)

t_start = time.time()
dma0.sendchannel.transfer(in_buffer0)
dma1.sendchannel.transfer(in_buffer1)
dma0.recvchannel.transfer(out_buffer_silicon)
dma0.sendchannel.wait()
dma1.sendchannel.wait()
dma0.recvchannel.wait()
t_stop = time.time()

in_buffer0.close()
in_buffer1.close()
out_buffer_silicon.close()

print("Hardware execution speed: ", t_stop - t_start)

# On Software

In [ ]:
%time numpy.add(in_buffer0, in_buffer1)